In [1]:
import tkinter as tk
from tkinter import filedialog, simpledialog, ttk
import cv2
from PIL import Image, ImageTk
import numpy as np
import matplotlib.pyplot as plt
import csv
import os

class ImageProcessingApp(tk.Frame):
    def __init__(self, master=None):
        super().__init__(master)
        self.master = master
        self.master.title("Image Processing App")
        
        # Center the window on the screen
        window_width = 600
        window_height = 600
        screen_width = self.master.winfo_screenwidth()
        screen_height = self.master.winfo_screenheight()

        # Calculate the position to center the window
        position_top = int(screen_height / 2 - window_height / 2)
        position_right = int(screen_width / 2 - window_width / 2)

        # Set window size and position
        self.master.geometry(f"{window_width}x{window_height}+{position_right}+{position_top}")
        self.master.configure(bg="#f0f0f5")

        # Initialize variables
        self.original_image = None
        self.current_image = None
        self.history_stack = []
        self.active_process = None
        self.color_index = 0

        self.csv_file = "user_details.csv"

        # Create widgets
        self.create_widgets()
        self.animate_heading()

        # Create CSV file if it doesn't exist
        if not os.path.exists(self.csv_file):
            with open(self.csv_file, mode='w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([ 
                    "Name", "Age", "Sex", "Height (cm)", 
                    "Weight (kg)", "Eye Color", "Hair Color", 
                    "State", "Charges"
                ])

    def create_widgets(self):
        self.heading_label = tk.Label(
            self.master,
            text="Image Preprocessing System",
            font=("Helvetica", 18, "bold"),
            bg="#4caf50",
            fg="white",
            pady=10
        )
        self.heading_label.pack(fill="x")

        self.canvas = tk.Canvas(self.master, bg="#f0f0f5")
        self.scrollbar = ttk.Scrollbar(self.master, orient="vertical", command=self.canvas.yview)
        self.scrollable_frame = ttk.Frame(self.canvas, style="TFrame")

        self.scrollable_frame.bind(
            "<Configure>",
            lambda e: self.canvas.configure(scrollregion=self.canvas.bbox("all"))
        )

        self.canvas.create_window((0, 0), window=self.scrollable_frame, anchor="n")
        self.canvas.configure(yscrollcommand=self.scrollbar.set)

        self.canvas.pack(side="left", fill="both", expand=True)
        self.scrollbar.pack(side="right", fill="y")

        load_button = tk.Button(self.scrollable_frame, text="Load Image", command=self.load_image, bg="#4caf50", fg="white", font=("Helvetica", 10, "bold"))
        load_button.grid(row=0, column=0, columnspan=3, padx=5, pady=5, sticky="ew")

        self.image_label = tk.Label(self.scrollable_frame, bg="#ffffff", relief="solid", borderwidth=1)
        self.image_label.grid(row=1, column=0, columnspan=3, pady=10, padx=10)

        self.size_label = tk.Label(self.scrollable_frame, text="Image Size: N/A", font=("Helvetica", 10), bg="#f0f0f5")
        self.size_label.grid(row=2, column=0, columnspan=3, pady=5)

        details_frame = ttk.Frame(self.scrollable_frame, style="TFrame")
        details_frame.grid(row=3, column=0, columnspan=3, pady=10, padx=10)

        fields = [
            ("Name:", "name_entry"),
            ("Age:", "age_entry"),
            ("Sex:", "sex_entry"),
            ("Height (CM):", "height_entry"),
            ("Weight (KG):", "weight_entry"),
            ("Eye Color:", "eye_color_entry"),
            ("Hair Color:", "hair_color_entry"),
            ("State:", "state_entry"),
            ("Charges:", "charges_entry")
        ]

        for idx, (label, attr) in enumerate(fields):
            tk.Label(details_frame, text=label, bg="#f0f0f5", font=("Helvetica", 10)).grid(row=idx, column=0, padx=5, pady=5, sticky="e")
            entry = tk.Entry(details_frame)
            entry.grid(row=idx, column=1, padx=5, pady=5, sticky="w")
            setattr(self, attr, entry)

        save_button = tk.Button(details_frame, text="Save Details", command=self.save_details, bg="#2196f3", fg="white", font=("Helvetica", 10, "bold"))
        save_button.grid(row=len(fields), column=0, columnspan=2, pady=10)

        menu_frame = ttk.Frame(self.scrollable_frame, style="TFrame")
        menu_frame.grid(row=4, column=0, columnspan=3, pady=10)

        self.menu_options = [
            ("Negative", 1),
            ("Contrast Stretching", 2),
            ("Resize", 3),
            ("Histogram", 4),
            ("Equalized Histogram", 5),
            ("Log Transformation", 6),
            ("Gamma Transformation", 7),
            ("Grayscale", 8)
        ]

        for idx, (text, value) in enumerate(self.menu_options):
            button = tk.Button(menu_frame, text=text, command=lambda v=value: self.toggle_process(v), bg="#ff9800", fg="white", font=("Helvetica", 10, "bold"))
            button.grid(row=idx // 4, column=idx % 4, padx=5, pady=5)

        download_button = tk.Button(self.scrollable_frame, text="Download Image", command=self.download_image, bg="#9c27b0", fg="white", font=("Helvetica", 10, "bold"))
        download_button.grid(row=5, column=0, columnspan=3, pady=10)

        exit_button = tk.Button(self.scrollable_frame, text="Exit", command=self.master.quit, bg="#f44336", fg="white", font=("Helvetica", 10, "bold"))
        exit_button.grid(row=6, column=0, columnspan=3, pady=10)

    def animate_heading(self):
        colors = ["#4caf50", "#ff5722", "#2196f3", "#9c27b0", "#ffeb3b"]
        self.color_index = (self.color_index + 1) % len(colors)
        self.heading_label.configure(bg=colors[self.color_index])
        self.master.after(500, self.animate_heading)

    def load_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg *.jpeg *.png")])
        if file_path:
            self.original_image = cv2.imread(file_path)
            self.original_image = cv2.cvtColor(self.original_image, cv2.COLOR_BGR2RGB)
            self.current_image = self.original_image.copy()
            self.history_stack = []
            self.active_process = None
            self.update_image_size()
            self.display_image()

    def update_image_size(self):
        if self.current_image is not None:
            height, width, _ = self.current_image.shape
            self.size_label.config(text=f"Image Size: {width} x {height}")

    def toggle_process(self, choice):
        if self.current_image is None:
            return

        if self.active_process == choice:
            if self.history_stack:
                self.current_image = self.history_stack.pop()
                self.active_process = None
                self.display_image()
        else:
            self.history_stack.append(self.current_image.copy())
            try:
                processed_img = self.process_image_impl(self.current_image.copy(), choice)
                self.current_image = processed_img
                self.active_process = choice
                self.display_image()
            except Exception as e:
                print(f"Error processing image: {e}")

    def process_image_impl(self, img, choice):
        if choice == 1:
            return 255 - img
        elif choice == 2:
            xp = [0, 64, 128, 192, 255]
            fp = [0, 16, 128, 240, 255]
            x = np.arange(256)
            table = np.interp(x, xp, fp).astype('uint8')
            return cv2.LUT(img, table)
        elif choice == 3:
            width = simpledialog.askinteger("Resize", "Enter new width:")
            height = simpledialog.askinteger("Resize", "Enter new height:")
            if width and height:
                return cv2.resize(img, (width, height))
            else:
                return img
        elif choice == 4:
            if len(img.shape) == 3:
                color = ('b', 'g', 'r')
                for i, col in enumerate(color):
                    histr = cv2.calcHist([img], [i], None, [256], [0, 256])
                    plt.plot(histr, color=col)
                plt.xlim([0, 256])
                plt.show()
            else:
                plt.hist(img.ravel(), 256, [0, 256])
                plt.show()
            return img
        elif choice == 5:
            if len(img.shape) == 3:
                img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
                img_yuv[:, :, 0] = cv2.equalizeHist(img_yuv[:, :, 0])
                return cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)
            else:
                return cv2.equalizeHist(img)
        elif choice == 6:
            img = img.astype(np.float32)
            max_val = np.max(img)
            if max_val == 0:
                return np.zeros_like(img, dtype=np.uint8)
            c = 255 / np.log(max_val + 1)
            log_transformed = c * np.log(img + 1)
            log_transformed = np.clip(log_transformed, 0, 255)
            return log_transformed.astype(np.uint8)
        elif choice == 7:
            gamma = simpledialog.askfloat("Gamma Transformation", "Enter gamma value:")
            if gamma is not None:
                return np.array(255 * (img / 255) ** gamma, dtype='uint8')
            else:
                return img
        elif choice == 8:
            if len(img.shape) == 3:
                return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            else:
                return img
        else:
            return img

    def save_details(self):
        details = [
            self.name_entry.get(),
            self.age_entry.get(),
            self.sex_entry.get(),
            self.height_entry.get(),
            self.weight_entry.get(),
            self.eye_color_entry.get(),
            self.hair_color_entry.get(),
            self.state_entry.get(),
            self.charges_entry.get()
        ]

        with open(self.csv_file, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(details)

        for entry in [self.name_entry, self.age_entry, self.sex_entry, self.height_entry, self.weight_entry, self.eye_color_entry, self.hair_color_entry, self.state_entry, self.charges_entry]:
            entry.delete(0, tk.END)

    def download_image(self):
        if self.current_image is None:
            return
        save_path = filedialog.asksaveasfilename(defaultextension=".jpg", filetypes=[("JPEG", "*.jpg"), ("PNG", "*.png")])
        if save_path:
            image = Image.fromarray(self.current_image)
            image.save(save_path)

    def display_image(self):
        if self.current_image is None:
            return
        img = Image.fromarray(self.current_image)
        img.thumbnail((400, 400))
        img_tk = ImageTk.PhotoImage(img)
        self.image_label.configure(image=img_tk)
        self.image_label.image = img_tk

if __name__ == "__main__":
    root = tk.Tk()
    app = ImageProcessingApp(master=root)
    app.mainloop()
